In [2]:
pip install plaid-python

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Installation des bibliothèques nécessaires
!pip install pandas numpy scikit-learn matplotlib seaborn nltk xgboost joblib plaid-python python-dotenv

  Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached contourpy-1.3.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.2 kB)
  Using cached pillow-11.1.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.1 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl (11.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 18.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 29.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 37.7 MB/s eta 0:00:0000:01
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# Bibliothèques standard
import os
import json
import re
import datetime

# Bibliothèques pour l'environnement
from dotenv import load_dotenv, dotenv_values

# Bibliothèques d'analyse de données
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Bibliothèques de traitement de texte
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Bibliothèques de machine learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib

# Bibliothèques Plaid
from plaid.api import plaid_api
from plaid.model.plaid_error import PlaidError
from plaid.model.products import Products
from plaid.model.link_token_create_request import LinkTokenCreateRequest
from plaid.model.country_code import CountryCode
from plaid.model.link_token_get_request import LinkTokenGetRequest
from plaid.model.accounts_get_request import AccountsGetRequest
from plaid.model.transactions_get_request import TransactionsGetRequest
from plaid.configuration import Configuration
from plaid.api_client import ApiClient
from plaid.model.item_public_token_exchange_request import ItemPublicTokenExchangeRequest
from plaid.model.sandbox_public_token_create_request import SandboxPublicTokenCreateRequest

## Load environment variables

In [5]:
load_dotenv() 
env = 'sand'
PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_SECRET = os.getenv("PLAID_SECRET_" + env)
PLAID_ENV = os.getenv("PLAID_ENV_" + env)

## Extracting and visualizing data from Plaid API

### 1 - Generate access token


In [6]:
# Initialize Plaid client configuration

configuration = Configuration(
    host=PLAID_ENV,
    api_key={
        'clientId': PLAID_CLIENT_ID,
        'secret': PLAID_SECRET,
    }
)

api_client = ApiClient(configuration)
client = plaid_api.PlaidApi(api_client)

request = LinkTokenCreateRequest(
    products=[Products('auth'), Products('transactions')],
    client_name="mycompany",
    country_codes=[CountryCode('FR')],
    language='fr',
    user={'client_user_id': 'user_12345'}
)

response = client.link_token_create(request)
link_token = response['link_token']

# Generate public token
pt_request = SandboxPublicTokenCreateRequest(
    institution_id='ins_117650',
    initial_products=[Products('transactions')]
)
pt_response = client.sandbox_public_token_create(pt_request)

# The generated public_token can now be
# exchanged for an access_token
exchange_request = ItemPublicTokenExchangeRequest(
    public_token=pt_response['public_token']
)
exchange_response = client.item_public_token_exchange(exchange_request)
access_token = exchange_response.access_token

### 2 - Load data using access token

#### Account data

In [16]:
# Retrieve account data using access token
request = AccountsGetRequest(
    access_token=access_token
)

try:
    accounts_response = client.accounts_get(request)
    accounts = accounts_response['accounts']
    accounts_list = [account.to_dict() for account in accounts]  # Convert to dictionary
    accounts_df = pd.DataFrame(accounts_list)
except Exception as e:
    print("Error retrieving accounts:", e)
    if hasattr(e, 'body'):
        try:
            error_response = json.loads(e.body)
            print(json.dumps(error_response, indent=2))
        except json.JSONDecodeError:
            print("Error decoding JSON response")
accounts_df.head(5)

,account_id,balances,mask,name,official_name,type,subtype,holder_category
0,rndX9e8GagF586gvb7xKuWAbWnvpNgF79Lljn,"{'available': 100.0, 'current': 110.0, 'limit'...",0000,Plaid Current Account,Plaid Standard Current Account,depository,checking,personal
1,zXQB9gyGZaFAB563jXWEtZMeZx1L3nflBDoR1,"{'available': 200.0, 'current': 210.0, 'limit'...",1111,Plaid Saving,Plaid Standard Interest Saving,depository,savings,personal
2,Bb6N9aeMnDFR9bg3v4mGhaoBa3pmZMS45qwdn,"{'available': None, 'current': 410.0, 'limit':...",3333,Plaid Credit Card,Plaid Diamond Credit Card,credit,credit card,NaN
3,3ApVd6lr7GI95qVvz8KoFnD3nx8qNEcZbGqkn,"{'available': None, 'current': 56302.06, 'limi...",8888,Plaid Mortgage,None,loan,mortgage,NaN
4,xn7L5pbGegFgb7D1nma6CXGMXB1NrDu6PmnxB,"{'available': 6009.0, 'current': 6009.0, 'limi...",9001,Plaid HSA,Plaid Cares Health Savings Account,depository,hsa,NaN


#### Transaction data

In [17]:
def serialize_response(obj):
    if isinstance(obj, (datetime.date, datetime.datetime)):
        return obj.isoformat()
    elif isinstance(obj, list):
        return [serialize_response(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: serialize_response(value) for key, value in obj.items()}
    else:
        return obj

start_date = datetime.datetime.now() - datetime.timedelta(days=30)
end_date = datetime.datetime.now()

request = TransactionsGetRequest(
    access_token=access_token,
    start_date=start_date.date(),
    end_date=end_date.date()
)

try:
    transactions_response = client.transactions_get(request)
    transactions = transactions_response['transactions']
    
    # Convert to dictionary and handle serialization
    transactions_list = [serialize_response(transaction.to_dict()) for transaction in transactions]
    transactions_df = pd.DataFrame(transactions_list)
except Exception as e:
    print("Error retrieving transactions:", e)
    if hasattr(e, 'body'):
        try:
            error_response = json.loads(e.body)
            print(json.dumps(error_response, indent=2))
        except json.JSONDecodeError:
            print("Error decoding JSON response")
transactions_df.head(5)

,account_id,account_owner,amount,authorized_date,authorized_datetime,category,category_id,check_number,counterparties,date,...,payment_meta,pending,pending_transaction_id,personal_finance_category,personal_finance_category_icon_url,transaction_code,transaction_id,transaction_type,unofficial_currency_code,website
0,rndX9e8GagF586gvb7xKuWAbWnvpNgF79Lljn,None,280.0,2025-04-09,None,"[Payment, Loan]",16003000,None,"[{'name': 'Loans 2 Go', 'type': 'merchant', 'w...",2025-04-09,...,"{'reference_number': None, 'ppd_id': None, 'pa...",False,None,"{'confidence_level': 'UNKNOWN', 'detailed': 'L...",https://plaid-category-icons.plaid.com/PFC_LOA...,None,6E8JdVr3ZeH4MzgNw8dpUzmVVvgn7Wu8PoZv4,special,None,None
1,rndX9e8GagF586gvb7xKuWAbWnvpNgF79Lljn,None,26.0,2025-04-09,None,"[Healthcare, Healthcare Services, Dentists]",14001012,None,"[{'name': 'Stobswell Dental Practice', 'type':...",2025-04-09,...,"{'reference_number': None, 'ppd_id': None, 'pa...",False,None,"{'confidence_level': 'UNKNOWN', 'detailed': 'M...",https://plaid-category-icons.plaid.com/PFC_MED...,None,d65e8q4knDcM49B3NvXyfgzWWDojMLTJnWKVk,place,None,None
2,rndX9e8GagF586gvb7xKuWAbWnvpNgF79Lljn,None,180.0,None,None,"[Recreation, Arts and Entertainment, Casinos a...",17001014,None,"[{'name': 'UniBet', 'type': 'merchant', 'websi...",2025-04-09,...,"{'reference_number': None, 'ppd_id': None, 'pa...",False,None,"{'confidence_level': 'UNKNOWN', 'detailed': 'E...",https://plaid-category-icons.plaid.com/PFC_ENT...,None,jnl3PxQ9qJF6QnWKrbagtwJ66ZAVE9H6RXdnb,place,None,unibet.com
3,rndX9e8GagF586gvb7xKuWAbWnvpNgF79Lljn,None,73.0,2025-04-07,None,None,None,None,"[{'name': 'Capital One', 'type': 'financial_in...",2025-04-07,...,"{'reference_number': None, 'ppd_id': None, 'pa...",False,None,"{'confidence_level': 'UNKNOWN', 'detailed': 'G...",https://plaid-category-icons.plaid.com/PFC_GEN...,None,xn7L5pbGegFgb7D1nma6CXGLLJ6e3wt6KGjrA,unresolved,None,None
4,rndX9e8GagF586gvb7xKuWAbWnvpNgF79Lljn,None,64.0,2025-04-07,None,"[Shops, Furniture and Home Decor]",19027000,None,"[{'name': 'IKEA', 'type': 'merchant', 'website...",2025-04-07,...,"{'reference_number': None, 'ppd_id': None, 'pa...",False,None,"{'confidence_level': 'UNKNOWN', 'detailed': 'H...",https://plaid-category-icons.plaid.com/PFC_HOM...,None,3ApVd6lr7GI95qVvz8KoFnDrrzVw45FZV1641,place,None,ikea.com


## Exploratory data analysis

In [18]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 28 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   account_id                          100 non-null    object 
 1   account_owner                       0 non-null      object 
 2   amount                              100 non-null    float64
 3   authorized_date                     90 non-null     object 
 4   authorized_datetime                 0 non-null      object 
 5   category                            97 non-null     object 
 6   category_id                         97 non-null     object 
 7   check_number                        0 non-null      object 
 8   counterparties                      100 non-null    object 
 9   date                                100 non-null    object 
 10  datetime                            0 non-null      object 
 11  iso_currency_code                   100 non-nu

In [12]:
transactions_df.describe()

,amount
count,100.000000
mean,63.297500
std,443.532181
min,-3000.000000
25%,22.000000
50%,43.500000
75%,73.000000
max,2078.500000


Create ML model to categorize transactions

Compare with plaid categorization